# METHOD 3: Ensemble with Research Paper Insights

## OBJECTIVE: Build an ensemble model incorporating insights from "Urban pluvial flooding prediction by machine learning approaches – a case study of Shenzhen city, China"

### TASK DESCRIPTION:
Create an ensemble that combines:
1. Multi-scale rainfall feature engineering (from paper)
2. Event regime classification (from paper's threshold approach)
3. Multiple base models (GNN-LSTM, Gradient Boosting, Random Forest)
4. Weighted averaging based on event characteristics

### KEY INSIGHTS FROM SHENZHEN PAPER TO IMPLEMENT:

1. MULTI-TEMPORAL RAINFALL FEATURES:
   Extract from the paper's methodology:
   
   R_d5: Max rainfall in any 5-min window
   R_d10: Max rainfall in any 10-min window
   R_d15: Max rainfall in any 15-min window
   R_d30: Max rainfall in any 30-min window
   R_d60: Max rainfall in any 60-min window
   R_d120: Max rainfall in any 120-min window
   R_d360: Max rainfall in any 360-min window
   R_d720: Max rainfall in any 720-min window
   R_d1440: Max rainfall in any 1440-min window (24 hours)
   
   Additionally compute:
   - Cumulative sums at same intervals
   - Rainfall intensity (mm/min)
   - Inter-event dry period (if applicable)

2. PCA-BASED EVENT CHARACTERIZATION:
   Following paper's Figure 3 approach:
   
   - Create feature matrix: [n_events × 9 rainfall features]
   - Apply PCA to reduce to 2-3 components
   - Visualize event space
   - Use PC scores as additional input features
   - Helps model understand "what type of storm is this?"

3. THRESHOLD-BASED REGIME DETECTION:
   Adapt paper's Table 2 thresholds to your data:
   
   Define regimes:
   - Low Risk: R_d60 < threshold_1
   - Medium Risk: threshold_1 ≤ R_d60 < threshold_2
   - High Risk: R_d60 ≥ threshold_2
   
   Find thresholds via:
   - Analyze when 1D nodes start to surcharge (overflow)
   - Analyze when 2D water levels exceed certain quantile (e.g., 75th percentile)
   - Train simple decision tree to find optimal split points
   
   Use regime as:
   - Categorical feature in models
   - Selector for specialized sub-models
   - Weighting factor in ensemble

4. ENSEMBLE ARCHITECTURE:

   Base Model 1: GNN-LSTM (from Method 2)
   - Strengths: Spatial structure, long-term dependencies
   - Weight: 0.4
   
   Base Model 2: Gradient Boosting (XGBoost/LightGBM)
   - Features: Multi-scale rainfall + static features + lags
   - Strengths: Non-linear patterns, feature interactions
   - Train separate models per node
   - Weight: 0.3
   
   Base Model 3: Random Forest Regressor
   - Features: Same as Gradient Boosting
   - Strengths: Robustness, less overfitting
   - Weight: 0.2
   
   Base Model 4: Regime-Specific Models
   - Train 3 specialized GNN-LSTMs (one per regime)
   - Route predictions based on detected regime
   - Weight: 0.1

5. FEATURE ENGINEERING PIPELINE:

   ```python
   def engineer_rainfall_features(rainfall_timeseries):
       """
       Based on Shenzhen paper's multi-temporal approach
       
       rainfall_timeseries: [timesteps, n_nodes]
       Returns: [timesteps, n_features]
       """
       features = {}
       
       # Define temporal windows (in timesteps, 5min each)
       windows = {
           'R_d5': 1,    # 5 min
           'R_d10': 2,   # 10 min
           'R_d15': 3,   # 15 min
           'R_d30': 6,   # 30 min
           'R_d60': 12,  # 1 hour
           'R_d120': 24, # 2 hours
           'R_d360': 72, # 6 hours
           'R_d720': 144 # 12 hours
       }
       
       for name, window in windows.items():
           # Rolling maximum
           features[f'{name}_max'] = rolling_max(rainfall_timeseries, window)
           
           # Rolling sum (accumulation)
           features[f'{name}_sum'] = rolling_sum(rainfall_timeseries, window)
       
       # Cumulative from start
       features['cumulative'] = np.cumsum(rainfall_timeseries, axis=0)
       
       # Intensity (rate of change)
       features['intensity'] = np.gradient(rainfall_timeseries, axis=0)
       
       # Time since rain started
       features['time_since_start'] = np.arange(len(rainfall_timeseries))
       
       return pd.DataFrame(features)
   ```

6. PCA EVENT EMBEDDINGS:

   ```python
   from sklearn.decomposition import PCA
   
   def create_event_embeddings(all_events_rainfall):
       """
       Extract PC scores for each event (Figure 3 in paper)
       """
       event_features = []
       
       for event_rain in all_events_rainfall:
           # Compute aggregate rainfall features per event
           R_d5_max = event_rain.max()
           R_d60_max = event_rain.rolling(12).max().max()
           R_d360_max = event_rain.rolling(72).max().max()
           # ... compute all 9 features
           
           event_features.append([R_d5_max, R_d10_max, ..., R_d1440_max])
       
       # PCA
       pca = PCA(n_components=3)
       pc_scores = pca.fit_transform(event_features)
       
       # These become input features for the model
       return pc_scores, pca
   ```

7. REGIME CLASSIFIER:

   ```python
   from sklearn.tree import DecisionTreeClassifier
   
   def train_regime_classifier(rainfall_features, water_levels):
       """
       Based on paper's threshold approach
       """
       # Define "High Risk" as when water levels exceed 75th percentile
       high_risk_threshold = np.percentile(water_levels, 75)
       labels = (water_levels > high_risk_threshold).astype(int)
       
       # Train classifier
       clf = DecisionTreeClassifier(max_depth=3)
       clf.fit(rainfall_features[['R_d60_max', 'R_d360_sum']], labels)
       
       # Extract thresholds from tree
       print(f"Regime thresholds: {clf.tree_.threshold}")
       
       return clf
   ```

8. ENSEMBLE WEIGHTING STRATEGY:

   Dynamic weighting based on event characteristics:
   
   ```python
   def ensemble_predict(models, X, event_pc_scores, regime):
       """
       Combine predictions with adaptive weights
       """
       predictions = {
           'gnn_lstm': models['gnn_lstm'].predict(X),
           'xgboost': models['xgboost'].predict(X),
           'rf': models['rf'].predict(X),
           'regime_specific': models[f'regime_{regime}'].predict(X)
       }
       
       # Base weights
       weights = {
           'gnn_lstm': 0.4,
           'xgboost': 0.3,
           'rf': 0.2,
           'regime_specific': 0.1
       }
       
       # Adjust based on event characteristics
       if event_pc_scores[0] > 2.0:  # Extreme event (from PCA)
           weights['gnn_lstm'] += 0.1  # GNN better for extremes
           weights['xgboost'] -= 0.1
       
       if regime == 'high_risk':
           weights['regime_specific'] += 0.15
           weights['rf'] -= 0.15
       
       # Normalize
       total = sum(weights.values())
       weights = {k: v/total for k, v in weights.items()}
       
       # Weighted average
       final_pred = sum(weights[k] * predictions[k] for k in weights)
       
       return final_pred
   ```

9. TRAINING PIPELINE:

   Step 1: Feature Engineering (Week 1)
   - Extract multi-scale rainfall features for all events
   - Compute PCA event embeddings
   - Train regime classifier
   - Save feature transformations
   
   Step 2: Train Base Models (Week 2)
   - Train GNN-LSTM (Method 2)
   - Train XGBoost per node with all features
   - Train Random Forest per node
   - Train 3 regime-specific GNN-LSTMs
   
   Step 3: Ensemble Optimization (Week 3)
   - Grid search over ensemble weights
   - Validate on cross-validation folds
   - Select best weight configuration
   - Retrain on full training set
   
   Step 4: Final Predictions (Week 3)
   - Generate predictions on test set
   - Apply ensemble weighting
   - Format for submission

10. CROSS-VALIDATION SPECIFIC TO THIS APPROACH:

    Stratified by:
    - Event regime (low/med/high risk)
    - PCA cluster (events in different regions of PC space)
    - Ensures validation set covers all storm types

### IMPLEMENTATION STEPS:

Day 1-2: Multi-scale rainfall feature engineering
Day 3: PCA event characterization
Day 4: Regime classifier training
Day 5-10: Train all base models
Day 11-12: Hyperparameter tuning for each model
Day 13-14: Ensemble weight optimization
Day 15-16: Cross-validation
Day 17-18: Error analysis and refinement
Day 19-20: Test predictions and submission

### CODE SKELETON:


In [ ]:
from sklearn.decomposition import PCA
from sklearn.tree import DecisionTreeClassifier
# from xgboost import XGBRegressor
# from sklearn.ensemble import RandomForestRegressor
import numpy as np
import pandas as pd

# Placeholder class for GNN-LSTM
class HybridGNN_LSTM:
    def fit(self, X, y):
        pass
    def predict(self, X):
        return np.zeros(len(X))

class RainfallFeatureEngineer:
    def transform(self, X, rainfall):
        # Placeholder
        return X

class EnsembleFloodPredictor:
    def __init__(self):
        self.models = {}
        self.feature_engineer = RainfallFeatureEngineer()
        self.pca = None
        self.regime_classifier = None
        self.ensemble_weights = None
    
    def _extract_event_features(self, rainfall):
        # Placeholder
        return np.zeros((len(rainfall), 10))
        
    def _train_regime_classifier(self, X, y):
        # Placeholder
        return DecisionTreeClassifier()

    def _optimize_weights(self, X, y):
        # Placeholder
        return {'model': 1.0}

    def _weighted_ensemble(self, predictions, pc_scores, regime):
        # Placeholder
        return list(predictions.values())[0]

    def fit(self, X_train, y_train, rainfall_train):
        # Step 1: Feature engineering
        X_enhanced = self.feature_engineer.transform(X_train, rainfall_train)
        
        # Step 2: PCA event embeddings
        event_features = self._extract_event_features(rainfall_train)
        self.pca = PCA(n_components=3).fit(event_features)
        pc_scores = self.pca.transform(event_features)
        
        # Step 3: Regime classification
        self.regime_classifier = self._train_regime_classifier(X_enhanced, y_train)
        regimes = self.regime_classifier.predict(X_enhanced)
        
        # Step 4: Train base models
        print("Training GNN-LSTM...")
        self.models['gnn_lstm'] = HybridGNN_LSTM()
        self.models['gnn_lstm'].fit(X_train, y_train)
        
        # print("Training XGBoost...")
        # self.models['xgboost'] = XGBRegressor(n_estimators=1000)
        # self.models['xgboost'].fit(X_enhanced, y_train)
        
        # print("Training Random Forest...")
        # self.models['rf'] = RandomForestRegressor(n_estimators=500)
        # self.models['rf'].fit(X_enhanced, y_train)
        
        print("Training regime-specific models...")
        for regime in ['low', 'medium', 'high']:
            # mask = (regimes == regime)
            self.models[f'regime_{regime}'] = HybridGNN_LSTM()
            # self.models[f'regime_{regime}'].fit(X_train[mask], y_train[mask])
        
        # Step 5: Optimize ensemble weights
        self.ensemble_weights = self._optimize_weights(X_train, y_train)
    
    def predict(self, X_test, rainfall_test):
        # Feature engineering
        X_enhanced = self.feature_engineer.transform(X_test, rainfall_test)
        
        # Event characterization
        event_features = self._extract_event_features(rainfall_test)
        pc_scores = self.pca.transform(event_features)
        regime = self.regime_classifier.predict(X_enhanced)
        
        # Get predictions from all models
        predictions = {}
        for name, model in self.models.items():
            predictions[name] = model.predict(X_test if 'gnn' in name else X_enhanced)
        
        # Weighted ensemble
        final_pred = self._weighted_ensemble(predictions, pc_scores, regime)
        
        return final_pred